# Locally interact wiht remote objects 
If we connect to the remote cluster from a notebook, we can interact with the remote objects we created in the training script in parallel 

In [12]:
import runhouse as rh 
num_nodes = 2
cluster_name = f"py-{num_nodes}-new-gcp"

env = rh.env(
    name="dask-env",
    load_from_den=False,
    reqs=[
        "dask-ml",
        "dask[distributed]",
        "dask[dataframe]",
        "gcsfs",
        "lightgbm",
    ],
)

cluster = rh.cluster(
    name=cluster_name,
    instance_type= 'CPU:2+',
    memory = "12+",
    num_nodes=num_nodes,
    provider="gcp",
    region = "us-east1",
    default_env=env,
).up_if_not()


INFO | 2024-11-13 00:14:27 | runhouse.resources.secrets.secret:285 | Saving config for /paul/ssh-sky-key to Den
INFO | 2024-11-13 00:14:27 | runhouse.resources.secrets.secret:301 | Saving secrets for /paul/ssh-sky-key to Vault
INFO | 2024-11-13 00:14:27 | runhouse.rns.rns_client:502 | Saving config for /paul/dask-env to RNS
INFO | 2024-11-13 00:14:28 | runhouse.rns.rns_client:502 | Saving config for /paul/py-2-new-gcp to RNS


Output()

INFO | 2024-11-13 00:14:31 | runhouse.resources.hardware.cluster:1759 | Running command on py-2-new-gcp: echo "hello"


In [3]:
# Use the return_model_details class on the remote object
trainer = cluster.get("my_trainer", remote = True, default = None)
print(trainer.return_model_details())

INFO | 2024-11-12 22:35:08 | runhouse.resources.hardware.ssh_tunnel:250 | Port 32300 is already in use. Trying next port.
INFO | 2024-11-12 22:35:08 | runhouse.resources.hardware.ssh_tunnel:250 | Port 32301 is already in use. Trying next port.
INFO | 2024-11-12 22:35:08 | runhouse.resources.hardware.ssh_tunnel:91 | Running forwarding command: ssh -T -L 32302:localhost:32300 -i ~/.ssh/sky-key -o Port=22 -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -o LogLevel=ERROR -o IdentitiesOnly=yes -o ExitOnForwardFailure=yes -o ServerAliveInterval=5 -o ServerAliveCountMax=3 -o ConnectTimeout=30s -o ForwardAgent=yes -o ControlMaster=auto -o ControlPath=/tmp/skypilot_ssh_6ba66db0/4b487bd908/%C -o ControlPersist=300s gcpuser@34.74.23.61
INFO | 2024-11-12 22:35:09 | runhouse.servers.http.http_client:439 | Calling my_trainer.return_model_details


INFO:httpx:HTTP Request: GET http://127.0.0.1:32302/logs/my_trainer/my_trainer_return_model_details@20241112@223509_408821/pickle "HTTP/1.1 200 OK"


INFO | 2024-11-12 22:35:09 | runhouse.servers.http.http_client:504 | Time to call my_trainer.return_model_details: 0.08 seconds
{'features': ['passenger_count', 'trip_distance', 'fare_amount', 'day', 'month', 'dayofweek', 'hour'], 'mse': np.float64(10.173834328554335), 'mae': np.float64(1.9945495515063427)}


In [4]:
# Run an inference on the remote model 
import numpy as np 
import dask.array as da
input_data = [3, 5, 110, 5, 11, 3, 6]

trainer.predict(input_data)

INFO | 2024-11-12 22:35:13 | runhouse.servers.http.http_client:439 | Calling my_trainer.predict


INFO:httpx:HTTP Request: GET http://127.0.0.1:32302/logs/my_trainer/my_trainer_predict@20241112@223513_267805/pickle "HTTP/1.1 200 OK"


------------
py-2-new-gcp
------------
dask.array<_predict_part, shape=(1,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
[10.87070356]
INFO | 2024-11-12 22:35:13 | runhouse.servers.http.http_client:504 | Time to call my_trainer.predict: 0.58 seconds


10.870703558582909

In [7]:
cluster.teardown()

INFO | 2024-11-12 22:40:33 | runhouse.resources.hardware.on_demand_cluster:618 | Tearing down cluster locally via Sky.
INFO | 2024-11-12 22:40:59 | runhouse.resources.secrets.secret:285 | Saving config for /paul/ssh-sky-key to Den
INFO | 2024-11-12 22:40:59 | runhouse.resources.secrets.secret:301 | Saving secrets for /paul/ssh-sky-key to Vault
INFO | 2024-11-12 22:40:59 | runhouse.rns.rns_client:502 | Saving config for /paul/dask-env to RNS
INFO | 2024-11-12 22:40:59 | runhouse.rns.rns_client:502 | Saving config for /paul/py-2-new-gcp to RNS
